# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning', 'R. E. Hviding']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

S. Kraus  ->  S. Kraus  |  ['S. Kraus']
J. Shi  ->  J. Shi  |  ['J. Shi']
S. Kumar  ->  S. Kumar  |  ['S. Kumar']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']
J. Wolf  ->  J. Wolf  |  ['J. Wolf']
K. Jahnke  ->  K. Jahnke  |  ['K. Jahnke']


T. Henning  ->  T. Henning  |  ['T. Henning']
Arxiv has 58 new papers today
          7 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/7 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2602.16009


extracting tarball to tmp_2602.16009...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.16227
extracting tarball to tmp_2602.16227...

 done.
Retrieving document from  https://arxiv.org/e-print/2602.16239


...

✔ → 138036:\section{Constraining power with fixed nuisance parameters}\label{sec:constraining_power_fixed_nuisance}
  ↳ 141823:end


Issues with the citations
entry with key cosmic_shear_review_2 has a duplicate keywords field
Retrieving document from  https://arxiv.org/e-print/2602.16646
extracting tarball to tmp_2602.16646...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[width=\hsize]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[angle=-90,width=3cm]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_d

/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:132: LatexWarning: attempting recovering figure empty.eps
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:707: LatexWarning: Could not find graphic \includegraphics[bb=10 20 100 300,clip]{empty.eps}
  warnings.warn(LatexWarning(f"Could not find graphic {k}"))


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16448-b31b1b.svg)](https://arxiv.org/abs/2602.16448) | **Euclid preparation. Impact of galaxy intrinsic alignment modelling choices on Euclid 3x2pt cosmology**  |
|| E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark> |
|*Appeared on*| *2026-02-19*|
|*Comments*| *23 pages, 16 figures, submitted to A&A*|
|**Abstract**|            The Euclid galaxy survey will provide unprecedented constraints on cosmology, but achieving unbiased results will require an optimal characterisation and mitigation of systematic effects. Among these, the intrinsic alignments (IA) of galaxies are one of the dominant contaminants of the weak lensing (WL) and galaxy-galaxy lensing (GGL) probes. In this work, we assess IA modelling choices for Euclid DR1 3x2pt analyses by comparing the performance of the two most commonly used IA models, nonlinear alignment (NLA) and tidal alignment tidal torquing (TATT), along with several variations. Our analyses combine three perspectives: i) the constraining power on the IA and cosmological parameters for each IA model, ii) the bias that results when the IA analysis model differs from the model used to generate the synthetic data vector, and iii) the degeneracies between IA and photometric redshift (photo-z) nuisance parameters. Among the IA models analysed, the redshift-dependent TATT model (zTATT) provides the most flexible description of IA, with a similar constraining power compared to simpler IA models, making it a well-motivated choice for Euclid DR1 3x2pt analyses.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16646-b31b1b.svg)](https://arxiv.org/abs/2602.16646) | **Two warm sub-Saturn mass planets identified from the TESS Full Frame Images**  |
|| F. I. Rojas, et al. -- incl., <mark>T. Henning</mark> |
|*Appeared on*| *2026-02-19*|
|*Comments*| *18 pages, 15 figures. Submitted to A&A*|
|**Abstract**|            Context. Characterization of warm giants is crucial to constrain giant planet formation and evolution. Measuring the mass and radius of these planets, combined with their moderated irradiation, allows us to estimate their planetary bulk composition, which is a key quantity to comprehend giant planet formation and structure. Aims. We present the discovery of two transiting warm giant planets orbiting solar-type stars from the Transiting Exoplanet Survey Satellite (TESS), which were characterized by further spectroscopic and photometric ground-based observations. Methods. We performed a joint analysis of photometric data with radial velocities to confirm and characterize TOI-883 b and TOI-899 b, two sub-Saturns orbiting solar-like stars. Results. TOI-883 b and TOI-899 b have masses of $0.123 \pm 0.012$ $M_J$ and $0.213 \pm 0.024$ $M_J$, radius of $0.604 \pm 0.028$ $R_J$ and $0.991 \pm 0.044$ $R_J$, periods of $10.06$ d and $12.85$ d and equilibrium temperature of $1086 \pm 19$ K and $1040 \pm 19$ K, respectively. Conclusions. While having similar masses, orbital periods and stellar host properties, these planets seem to have different internal compositions, which could point to distinct formation histories. Both planets are suitable targets for atmospheric studies to further constrain formation scenarios of planets in the Neptune-Saturn mass range         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16009-b31b1b.svg)](https://arxiv.org/abs/2602.16009) | **CHARA Array Delay Lines: Upgrades, Performance and Future Directions**  |
|| N. Anugu, et al. -- incl., <mark>S. Kraus</mark> |
|*Appeared on*| *2026-02-19*|
|*Comments*| *Accepted for publication in SPIE Journal of Astronomical Telescopes, Instruments, and Systems (JATIS). 33 pages, 17 figures*|
|**Abstract**|            Long baseline optical and infrared interferometric arrays achieve high angular resolution and enable detailed astrophysical measurements. Interferometers have enabled observations of stars at various stages of evolution, as well as studies of binary stars, circumstellar disks, and active galactic nuclei. The CHARA Array is a long-baseline interferometric array at the Mount Wilson Observatory, USA. At the core of CHARA operations are the delay lines, which equalize the optical path length for all telescopes as the Earth rotates and compensate for optical path variations induced by atmospheric turbulence. We report recent upgrades and performance of the CHARA Array optical delay lines for high-precision interferometric observations. The legacy system had been operational for over two decades, and it was increasingly difficult to acquire replacement parts. Beginning in mid-2021, the control system underwent a major upgrade, replacing the aging VME-based architecture with a modern hybrid FPGA and Linux-based system; this modernization continued through the end of 2024. We describe hardware/software changes, the servo architecture, and lab/on-sky performance. The upgraded system achieves residual delay line cart tracking errors of $\sim12$~nm, the same level as the legacy system, and a control bandwidth of 100-130~Hz, allowing fringe tracking across the R, H, and K bands. Initial commissioning revealed key issues such as metrology time-tick jitter and vibration-induced visibility loss, which were diagnosed and resolved. We note ongoing and future efforts to extend baselines up to 1~km and support advanced observing modes such as dual-field interferometry and nulling. This paper is a reference for current and future use of the CHARA Array and for next-generation instrument design.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16227-b31b1b.svg)](https://arxiv.org/abs/2602.16227) | **The Enigmatic Type Icn Supernova 2024abvb Located ~22 kpc from Its Host Galaxy**  |
|| <mark>J. Shi</mark>, et al. |
|*Appeared on*| *2026-02-19*|
|*Comments*| *25 pages, 18 figures, 4 tables. Submitted to PASA*|
|**Abstract**|            We report multiwavelength observations of the highly offset (~22.4 kpc) SN 2024abvb, the sixth Type Icn supernova to date. With a peak magnitude of Mr = -19.55 +/- 0.11 mag, it is among the most luminous in the existing sample and shows similar colours and decline rates to other SNe Icn. The early optical spectra show a blue continuum with narrow C II features (vFWHM ~ 2000 km s^-1), consistent with a typical wind velocity of a Wolf-Rayet star. The absence of C III lambda 5696 emission at the time of explosion is consistent with a Type Ibn supernova; however, the lack of narrow He lines in both the optical and near-infrared spectra supports a SNe Icn classification. Unlike the majority of SNe Icn, we do not detect broad features in the late-time (7-21 days relative to o-band peak) spectral phase of SN 2024abvb. Semi-analytical modelling of the light curves shows that it can be reproduced by ~2.6 Msun of SN ejecta interacting with ~0.3 Msun of circumstellar material (CSM), both larger than other SNe Icn but consistent with rapidly evolving SNe Ibn. The metallicity at the SN location is significantly lower than the global metallicity of its host galaxy, suggesting that line-driven mass loss required to strip the progenitor of its H and He envelopes was likely inefficient. We estimate the star-formation-rate history at the location of SN 2024abvb and find that it lies at the bottom ~5th percentile among SESNe hosts, inconsistent with a Wolf-Rayet progenitor. Based on its spectral features, local and host environment properties, and host-galaxy offset, we favour an 8-10 Msun star stripped by a compact companion as the progenitor, with a sufficient runaway velocity to reach the observed offset.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16239-b31b1b.svg)](https://arxiv.org/abs/2602.16239) | **Propagation Characteristics of the April 21, 2023 CME**  |
|| <mark>S. Kumar</mark>, N. Srivastava, P. Banerjee, N. Gopalswamy |
|*Appeared on*| *2026-02-19*|
|*Comments*| *Accepted for publication in The Journal of Astrophysics and Astronomy*|
|**Abstract**|            Accurate estimation of propagation characteristics of coronal mass ejections (CMEs) is crucial for predicting their geoeffectiveness. Stereoscopic techniques to study the kinematics of CMEs generally have been carried out using remote sensing observations from three viewpoints, i.e. STEREO-A, STEREO-B, and SOHO. Since the loss of STEREO-B in 2014, stereoscopic reconstruction of CMEs has been restricted to the observations from only two viewpoints, i.e., STEREO-A and SOHO. When the angle of separation between STEREO-A and SOHO is small, it leads to larger uncertainties in the CME kinematics derived using stereoscopic techniques. In this paper, we demonstrate how this limitation can be addressed and how uncertainties in the estimation of CME kinematics and propagation direction can be reduced. For this purpose, we selected the CME of April 21, 2023, which was observed by two spacecraft, i.e. STEREO-A and SOHO, separated by a small 10 degree angle. Using the Graduated Cylindrical Shell (GCS) model on the remote-sensing observations near the Sun and the Advanced Drag-Based Model (ADBM) in the heliosphere, we estimated the arrival time of the CME at different locations in the heliosphere and compared it with the actual arrival time obtained from the in-situ measurements taken by three spacecraft, BepiColombo, STEREO-A and Wind. Our analysis reveals a directional uncertainty of approx 20 degree from observations from two viewpoints. These uncertainties significantly affect the arrival-time prediction of the CME. We consider the actual chronology of CME arrival times at STEREO-A and Wind as critical parameters to constrain the direction of propagation, which serves as a key input in the ADBM. The chronology of arrival of the CME ejecta at STEREO-A, which is 4.5 hrs earlier than at Wind, proved essential for resolving directional ambiguities in the GCS reconstruction model         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16409-b31b1b.svg)](https://arxiv.org/abs/2602.16409) | **A fast radio burst cyclone in technicolour: evidence of plasma lensing**  |
|| P. A. Uttarkar, et al. -- incl., <mark>Y. Wang</mark> |
|*Appeared on*| *2026-02-19*|
|*Comments*| *18 figures, 3 tables*|
|**Abstract**|            Fast radio bursts (FRBs) are bright, energetic, radio pulses of extragalactic origin. A dichotomy has emerged in the population: some produce repeat bursts, while the majority do not. Most repeating sources only show rare repetitions, and none have been studied extensively over the wide bandwidths necessary to disentangle the physical processes that produce emission from distortions to bursts caused by intervening ionised gas. Here we present radio observations of the most active repeating source, FRB 20240114A. Using an ultrawideband receiving system, we have detected 5526 repetitions, revealing an extreme spectral and temporal variability in the burst emission. The bursts exhibit longer-term broadband variations in central emission frequency over multiple months, and narrowband bursts that have correlations in central frequencies on time scales of milliseconds to minutes. The spectral and temporal properties are consistent with the source undergoing magnification by foreground plasma lenses, potentially embedded in a turbulent circumsource medium. This extreme example highlights the role of plasma lenses in the observed properties of burst emission and can explain the diversity in activity and energetics of the entire FRB population.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2602.16415-b31b1b.svg)](https://arxiv.org/abs/2602.16415) | **The Astronomical Telescope of the University of Stuttgart (ATUS): Development, Optimization, and Lessons Learned**  |
|| K. Schindler, <mark>J. Wolf</mark>, A. Krabbe |
|*Appeared on*| *2026-02-19*|
|*Comments*| *44 pages, 30 figures, 8 tables. This is the Accepted Manuscript version of an article accepted for publication in the Publications of the Astronomical Society of the Pacific. IOP Publishing Ltd is not responsible for any errors or omissions in this version of the manuscript or any version derived from it*|
|**Abstract**|            ATUS, the Astronomical Telescope of the University of Stuttgart, is a fully remote-controlled 0.6 m f/8.17 Ritchey-Chrétien telescope optimized for high-cadence, high-fidelity photometry of transient sources. Observations are time-referenced with very high accuracy and precision, making it an ideal platform for time-domain astronomy and space situational awareness. Initially conceived to support instrument developments and operations of SOFIA, the Stratospheric Observatory for Infrared Astronomy, it evolved into a scientific instrument for various use cases in instrument development, astronomical research, and teaching. This paper presents an overview of its development and optimization to achieve diffraction-limited images and highly accurate pointing and tracking, even at high speeds. The findings and lessons learned are universally applicable to other telescopes that are currently at the planning stage, or where similar issues might be encountered.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2602.16448/./Figures/IA_values_FS.png', 'tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_IA_params.png', 'tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_cosmo_all.png', 'tmp_2602.16448/./Figures/DV_NLA_DV_TATT_different_probes.png']
copying  tmp_2602.16448/./Figures/IA_values_FS.png to _build/html/
copying  tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_IA_params.png to _build/html/
copying  tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_cosmo_all.png to _build/html/
copying  tmp_2602.16448/./Figures/DV_NLA_DV_TATT_different_probes.png to _build/html/
exported in  _build/html/2602.16448.md
    + _build/html/tmp_2602.16448/./Figures/IA_values_FS.png
    + _build/html/tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_IA_params.png
    + _build/html/tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_cosmo_all.png
    + _build/html/tmp_2602.16448/./Figures/DV_NLA_DV_TATT_different_probes.png
found figures ['', '', 'tmp_2602.16646/./img/phasedB.png', 'tmp_2602.16646/./img/phasedA.png']
file

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\todo}[1]{{\sffamily\itshape\tiny[\ignorespaces #1]}}$
$\newcommand{\orcid}[1]$
$\newcommand\itt{#1}$
$\newcommand\itc{#1}$
$\newcommand\dnt{#1}$</div>



<div id="title">

# $\Euclid$ preparation: Impact of galaxy intrinsic alignment modelling choices on $\Euclid$ $3\times 2$ pt cosmology

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.16448-b31b1b.svg)](https://arxiv.org/abs/2602.16448)<mark>Appeared on: 2026-02-19</mark> -  _23 pages, 16 figures, submitted to A&A_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>K. Jahnke</mark>

</div>
<div id="abstract">

**Abstract:** The $\Euclid$ galaxy survey will provide unprecedented constraints on cosmology, but achieving unbiased results will require an optimal characterisation and mitigation of systematic effects. Among these, the $\gls{ia}$ of galaxies are one of the dominant contaminants of the $\gls{wl}$ and $\gls{ggl}$ probes. In this work, we assess $\gls{ia}$ modelling choices for $\Euclid$ DR1 $3\times 2$ pt analyses by comparing the performance of the two most commonly used $\gls{ia}$ models, $\gls{nla}$ and $\gls{tatt}$ , along with several variations. Our analyses combine three perspectives: i) the constraining power on the $\gls{ia}$ and cosmological parameters for each $\gls{ia}$ model, ii) the bias that results when the $\gls{ia}$ analysis model differs from the model used to generate the synthetic data vector, and iii) the degeneracies between $\gls{ia}$ and $\gls{photo-z}$ nuisance parameters. Among the $\gls{ia}$ models analysed, the redshift-dependent TATT model ( $\tattz$ ) provides the most flexible description of $\gls{ia}$ , with a similar constraining power compared to simpler $\gls{ia}$ models, making it a well-motivated choice for $\Euclid$ DR1 $3\times 2$ pt analyses.

</div>

<div id="div_fig1">

<img src="tmp_2602.16448/./Figures/IA_values_FS.png" alt="Fig3" width="100%"/>

**Figure 3. -** \gls{ia} constraints for the measured Flagship $C_{\ell}$ described in Sect. \ref{sec:IA_values_FS} when using the \tattz(tan) and the \nlaz(blue) \gls{ia} models. The dashed lines correspond to the maximum of the posterior distributions of the \tattz parameters, which are used to generate the synthetic \glspl{dv} in Sect. \ref{sec:generation_synt_DVs}. (*fig:IA_values_FS*)

</div>
<div id="div_fig2">

<img src="tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_IA_params.png" alt="Fig9.1" width="50%"/><img src="tmp_2602.16448/./Figures/scale_cuts_NLA_TATT_cosmo_all.png" alt="Fig9.2" width="50%"/>

**Figure 9. -** \gls{ia}*(left)* and cosmological parameters *(right)* constraints for \nlaz(blue) and \tattz(tan) at different scale cuts in the \gls{wl}$C_{\ell}$, $k_{\rm{max}}=1 h\rm{Mpc}^{-1}$(unfilled) and $k_{\rm{max}}=3 h\rm{Mpc}^{-1}$(filled). The dashed lines indicate the fiducial values of the parameters. The tan contours on the right plot are not well distinguished because they overlap with the blue ones. (*fig:TATT_NLA_scale_cuts*)

</div>
<div id="div_fig3">

<img src="tmp_2602.16448/./Figures/DV_NLA_DV_TATT_different_probes.png" alt="Fig4" width="100%"/>

**Figure 4. -** Constraints on $S_{8}$ and $\Omega_{\rm{m}}$ for \nlaz(filled contours) and \tattz(unfilled contours), and for different combinations of cosmological probes: WL (tan), $2\times 2$ pt (blue), and $3\times 2$ pt (black). The dashed lines indicate the fiducial values of the parameters. (*fig:constraining_power_different_probes*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.16448"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\feh}{\ensuremath{{\rm[Fe/H]}}}$
$\newcommand{\teff}{\ensuremath{T_{\rm eff}}}$
$\newcommand{\teq}{\ensuremath{T_{\rm eq}}}$
$\newcommand{\logg}{\ensuremath{\log{g}}}$
$\newcommand{\lightkurve}{\texttt{lightkurve}}$
$\newcommand{\astrocut}{\texttt{astrocut}}$
$\newcommand{\astroquery}{\texttt{astrocut}}$
$\newcommand{\tesseract}{\texttt{tesseract}}$
$\newcommand{\juliet}{\texttt{juliet}}$
$\newcommand{\zaspe}{\texttt{ZASPE}}$
$\newcommand{\ceres}{\texttt{CERES}}$
$\newcommand{\tess}{\textit{TESS}}$
$\newcommand{\vsini}{\ensuremath{v \sin{i}}}$
$\newcommand{\kms}{\ensuremath{{\rm km s^{-1}}}}$
$\newcommand{\mjup}{\ensuremath{{\rm M_{J}}}}$
$\newcommand{\mearth}{\ensuremath{{\rm M}_{\oplus}}}$
$\newcommand{\mplanet}{\ensuremath{{\rm M_P}}}$
$\newcommand{\rjup}{\ensuremath{{\rm R_J}}}$
$\newcommand{\rplanet}{\ensuremath{{\rm R_P}}}$
$\newcommand{\rstar}{\ensuremath{{\rm R}_{\star}}}$
$\newcommand{\mstar}{\ensuremath{{\rm M}_{\star}}}$
$\newcommand{\lstar}{\ensuremath{{\rm L}_{\star}}}$
$\newcommand{\rsun}{\ensuremath{{\rm R}_{\odot}}}$
$\newcommand{\msun}{\ensuremath{{\rm M}_{\odot}}}$
$\newcommand{\lsun}{\ensuremath{{\rm L}_{\odot}}}$
$\newcommand{\rhoplanet}{\ensuremath{{\rm \rho_P}}}$
$\newcommand{\gccm}{\ensuremath{\mathrm{g} \mathrm{cm}^{-3}}}$
$\newcommand{\PpA}{\ensuremath{10.057716 	 	 ^{+0.000016} 	  _{-0.000016}}}$
$\newcommand{\tcpA}{\ensuremath{2458466.4743	 ^{+0.0017} 	  _{-0.0018}}}$
$\newcommand{\muHARPSA}{\ensuremath{16.6492 	 	 ^{+0.0009} 	  _{-0.0009}}}$
$\newcommand{\muFEROSA}{\ensuremath{16.6149 	 	 ^{+0.0022} 	  _{-0.0023}}}$
$\newcommand{\sigmawrvFEROSA}{\ensuremath{0.0100 	 	 ^{+0.0023} 	  _{-0.0019}}}$
$\newcommand{\sigmawrvHARPSA}{\ensuremath{0.0030 	 	 ^{+0.0009} 	  _{-0.0007}}}$
$\newcommand{\apA}{\ensuremath{19.54 	 	 ^{+0.23} 	  _{-0.32}}}$
$\newcommand{\rhoA}{\ensuremath{1394 	 	 ^{+50} 	  _{-67}}}$
$\newcommand{\qaTESSA}{\ensuremath{0.66 	 	 ^{+0.24} 	  _{-0.34}}}$
$\newcommand{\GPrhoTESSLCA}{\ensuremath{0.100 	 	 ^{+0.026} 	  _{-0.023}}}$
$\newcommand{\GPsigmaTESSLCA}{\ensuremath{269 	 	 ^{+27} 	  _{-26}}}$
$\newcommand{\KpA}{\ensuremath{0.0124 	 	 ^{+0.0005} 	  _{-0.0005}}}$
$\newcommand{\sigmawTESSA}{\ensuremath{487 	 	 ^{+15} 	  _{-15}}}$
$\newcommand{\mfluxTESSA}{\ensuremath{-0.000010 	 	 ^{+0.000032} 	  _{-0.000032}}}$
$\newcommand{\rapA}{\ensuremath{0.593 	 	 ^{+0.083} 	  _{-0.168}}}$
$\newcommand{\rbpA}{\ensuremath{0.119 	 	 ^{+0.005} 	_{-0.008}}}$
$\newcommand{\qapA}{\ensuremath{0.71 	 	 ^{+0.21} 	  _{-0.39}}}$
$\newcommand{\qbpA}{\ensuremath{0.41 	 	 ^{+0.36} 	_{-0.29}}}$
$\newcommand{\ppA}{\ensuremath{0.0597 	 	 ^{+0.0027} 	  _{-0.0040}}}$
$\newcommand{\bpA}{\ensuremath{0.4919 	 	 ^{+0.1034} 	  _{-0.2095}}}$
$\newcommand{\epA}{\ensuremath{0}}$
$\newcommand{\incpA}{\ensuremath{88.66 	 	 ^{+0.56} 	  _{-0.27}}}$
$\newcommand{\mstA}{\ensuremath{0.956 _{-0.051}^{+0.054}}}$
$\newcommand{\rstA}{\ensuremath{0.988 _{-0.012}^{+0.012}}}$
$\newcommand{\rhostA}{\ensuremath{1.40   _{-0.14}^{+0.15}}}$
$\newcommand{\lstA}{\ensuremath{0.934 _{-0.031}^{+0.043}}}$
$\newcommand{\ageA}{\ensuremath{6.5   _{-2.1}^{+2.0}}}$
$\newcommand{\AvA}{\ensuremath{0.075   _{-0.049}^{+0.064}}}$
$\newcommand{\aA}{\ensuremath{0.0895_{-0.0021}^{+0.0022}}}$
$\newcommand{\mplA}{\ensuremath{0.123\pm 0.012}}$
$\newcommand{\rplA}{\ensuremath{0.604\pm 0.028}}$
$\newcommand{\teqA}{\ensuremath{1086\pm 19}}$
$\newcommand{\PpB}{\ensuremath{12.846185	 	 ^{+0.000008} 	  _{-0.000008}}}$
$\newcommand{\mstB}{\ensuremath{0.972 _{-0.053}^{+0.054}}}$
$\newcommand{\rstB}{\ensuremath{1.086 _{-0.012}^{+0.013}}}$
$\newcommand{\rhostB}{\ensuremath{1.07   _{-0.10}^{+0.11}}}$
$\newcommand{\lstB}{\ensuremath{1.113 _{-0.052}^{+0.054}}}$
$\newcommand{\ageB}{\ensuremath{8.4   _{-2.0}^{+2.0}}}$
$\newcommand{\AvB}{\ensuremath{0.176  _{-0.069}^{+0.073}}}$
$\newcommand{\aB}{\ensuremath{0.1063\pm 0.0026}}$
$\newcommand{\mplB}{\ensuremath{0.213\pm 0.024}}$
$\newcommand{\rplB}{\ensuremath{0.991\pm 0.044}}$
$\newcommand{\teqB}{\ensuremath{1040\pm 19}}$
$\newcommand{\teffA}{\ensuremath{5697 \pm 80}}$
$\newcommand{\loggA}{\ensuremath{4.429 \pm 0.022}}$
$\newcommand{\fehA}{\ensuremath{0.02 \pm 0.04}}$
$\newcommand{\vsiniA}{\ensuremath{2.53 \pm 0.30}}$
$\newcommand{\teffB}{\ensuremath{5696 \pm 80}}$
$\newcommand{\loggB}{\ensuremath{4.355 \pm 0.022}}$
$\newcommand{\fehB}{\ensuremath{0.10 \pm 0.04}}$
$\newcommand{\vsiniB}{\ensuremath{3.65 \pm 0.30}}$
$\newcommand{\plnameA}{TOI-883b}$
$\newcommand{\stnameA}{TOI-883}$
$\newcommand{\plnameB}{TOI-899b}$
$\newcommand{\stnameB}{TOI-899}$
$\newcommand{\rhopkep}{\ensuremath{1.154 \pm 0.045 }}$</div>



<div id="title">

# Two warm sub-Saturn mass planets identified from the TESS FullFrame Images$\thanks{This paper includes data gathered with the 6.5 meter Magellan Telescopes located at Las Campanas Observatory, Chile}$

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2602.16646-b31b1b.svg)](https://arxiv.org/abs/2602.16646)<mark>Appeared on: 2026-02-19</mark> -  _18 pages, 15 figures. Submitted to A&A_

</div>
<div id="authors">

F. I. Rojas, et al. -- incl., <mark>T. Henning</mark>

</div>
<div id="abstract">

**Abstract:** Characterization of warm giants is crucial to constrain giant planet formation and evolution. Measuring the mass and radius of these planets, combined with their moderated irradiation, allows us to estimate their planetary bulk composition, which is a key quantity to comprehend giant planet formation and structure. We present the discovery of two transiting warm giant planets orbiting solar-type stars from theTransiting Exoplanet Survey Satellite (TESS), which were characterized by further spectroscopic and photometric ground-based observations. We performed a joint analysis of photometric data with radial velocities to confirm and characterize $\plnameA$ and $\plnameB$ , two sub-Saturns orbiting solar-like stars. $\plnameA$ and $\plnameB$ have masses of $\mplA$ $\mjup$ and $\mplB$ $\mjup$ , radius of $\rplA$ $\rjup$ and $\rplB$ $\rjup$ , periods of 10.06 d and 12.85 d and equilibrium temperature of $\teqA$ K and $\teqB$ , respectively. While having similar masses, orbital periods and stellar host properties, these planets seem to have different internal compositions, which could point to distinct formation histories. Both planets are suitable targets for atmospheric studies to further constrain formation scenarios of planets in the Neptune-Saturn mass range.

</div>

<div id="div_fig1">

<img src="" alt="Fig5.1" width="50%"/><img src="" alt="Fig5.2" width="50%"/>

**Figure 5. -** Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Vibrational stability equation of state
               $S_{\mathrm{vib}}(\lg e, \lg \rho)$.
               $>0$ means vibrational stability.
              Nonlinear Model ResultsNonlinear Model ResultsSpectral types and photometry for stars in the
  region.Spectral types and photometry for stars in the
  region.List of nearby SNe used in this work.Summary for ISOCAM sources with mid-IR excess
(YSO candidates).Summary for ISOCAM sources with mid-IR excess
(YSO candidates). Sample stars with absolute magnitudecontinued. Sample stars with absolute magnitudecontinued.Complexes characterisation.Line data and abundances ...Continued. (*FigVibStab*)

</div>
<div id="div_fig2">

<img src="tmp_2602.16646/./img/phasedB.png" alt="Fig7" width="100%"/>

**Figure 7. -** Transit data observed for $\plnameB$. The phase folded transit is plotted for both TESS long and short cadence. For CHAT and El Sauce, a single transit event is plotted. (*fig:phasedB*)

</div>
<div id="div_fig3">

<img src="tmp_2602.16646/./img/phasedA.png" alt="Fig6" width="100%"/>

**Figure 6. -** Photometric observations and transits for $\plnameA$. In the case of TESS, both light curves are phase folded. CHAT and both from Las Cumbres Observatory are follow-up data that observed a single transit. (*fig:phasedA*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2602.16646"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

115  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

2  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
